In [11]:
import os
import sys
sys.path.insert(0, '/global/homes/k/kovacs/gcr-catalogs_diffsky_v0.1')
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import astropy.constants
from astropy.constants import iaudata
from astropy import units as u
import GCRCatalogs

In [12]:
gc = GCRCatalogs.load_catalog('diffsky_v0.1_p3765_vsmall')
native_quantities = sorted(gc.list_all_native_quantities())
from lsstdesc_diffsky.constants import MAH_PNAMES, MS_U_PNAMES, Q_U_PNAMES
print(MAH_PNAMES, MS_U_PNAMES, Q_U_PNAMES)
DUST_PNAMES = ['dust_Eb', 'dust_delta', 'dust_Av']
print(gc.cosmology)

['diffmah_logmp_fit', 'diffmah_mah_logtc', 'diffmah_early_index', 'diffmah_late_index'] ['diffstar_u_lgmcrit', 'diffstar_u_lgy_at_mcrit', 'diffstar_u_indx_lo', 'diffstar_u_indx_hi', 'diffstar_u_tau_dep'] ['diffstar_u_qt', 'diffstar_u_qs', 'diffstar_u_q_drop', 'diffstar_u_q_rejuv']
FlatLambdaCDM(H0=71.0 km / (Mpc s), Om0=0.2648, Tcmb0=0.0 K, Neff=3.04, m_nu=None, Ob0=0.0448)


## Load SSP data

In [13]:
from dsps import calc_rest_sed_sfh_table_met_table
DSPS_data_dir = '/global/cfs/cdirs/lsst/groups/CS/Catalog_5000/dsps_data/FSPS_ssp_data/'
DSPS_data_fn = 'ssp_data_fsps_v3.2_lgmet_age.h5'
from dsps.data_loaders.load_ssp_data import load_ssp_templates
ssp_data = load_ssp_templates(fn=os.path.join(DSPS_data_dir, DSPS_data_fn))
print(ssp_data._fields)

('ssp_lgmet', 'ssp_lg_age_gyr', 'ssp_wave', 'ssp_flux')


In [21]:
#Compute times
T0 = gc.cosmology.age(0.0).value
LGT0 = np.log10(T0)
times = np.linspace(0.1, T0, 100)

In [14]:
#Load catalog data and select subsample of galaxies
cat_data = gc.get_quantities(MAH_PNAMES + MS_U_PNAMES + Q_U_PNAMES + ['redshift', 'mag_r', 'mag_i'])

In [15]:
cat_data['r-i'] = cat_data['mag_r'] -  cat_data['mag_i']
red_mask = (cat_data['r-i'] > 1.5) &  (cat_data['redshift'] < 1.0)
print(np.count_nonzero(red_mask))
blue_mask = (cat_data['r-i'] < 0.0) &  (cat_data['redshift'] < 1.0)
print(np.count_nonzero(blue_mask))

746
337


# Define function to return galaxy SED

In [16]:
import galsim

def evaluate_rest_sed(galaxy_data, ssp_data, gal_sfh_table, t_table):
    """Return the rest-frame SED of the galaxy."""
    zz = galaxy_data['redshift_true']
    t_obs = gc.cosmology.lookback_time(zz)
    
    gal_lgmet = galaxy_data['lg_met_mean']
    gal_lgmet_scatter = gal_lgmet_scatter['lg_met_scatter']
    
    sed_args = (t_table,
                gal_sfh_table,
                gal_lgmet,
                gal_lgmet_scatter,
                ssp_data.ssp_lgmet,
                ssp_data.ssp_lg_age_gyr,
                ssp_data.ssp_flux,
                t_obs,
                )

    rest_sed = calc_rest_sed_sfh_table_lognormal_mdf(*sed_args)
 
    #rest_sed = calc_rest_sed_sfh_table_met_table(*sed_args)  #with metallicity history
                            
    dust_params = np.array([galaxy_pars[_] for _ in DUST_PNAMES])
    atten = _get_att_curve_kern(ssp_data.ssp_wave, dust_params)
    
    Lnu = np.array(rest_sed*atten)  # Lsun/Hz

    # Collect conversion factors = L_sun*ergs_per_joule*clight/4./pi/dl**2
    ERG_PER_JOULE = 1e7
    CM_PER_MPC = 3.085677581491367e+24
    clight = astropy.constants.c.value*1e2  # cm/s
    dl = gc.cosmology.luminosity_distance(zz).value*CM_PER_MPC
    factor = iaudata.L_sun.value*ERG_PER_JOULE*clight/4/np.pi/dl**2

    wl = self.ssp_data.ssp_wave*1e-8  # convert from Angstroms to cm
    Flambda = Lnu*factor/wl**2
    lut = galsim.LookupTable(ssp_data.ssp_wave, Flambda)
    return galsim.SED(lut, wave_type=u.Angstrom, flux_type='flambda')

In [17]:
mah_params = np.array([cat_data[_] for _ in MAH_PNAMES]).T
ms_u_params = np.array([cat_data[_] for _ in MS_U_PNAMES]).T
q_u_params = np.array([cat_data[_] for _ in Q_U_PNAMES]).T
#q_u_params = assemble_param_arrays(cat_data, Q_U_PNAMES)
print(mah_params.shape, ms_u_params.shape, q_u_params.shape)

(19333976, 4) (19333976, 5) (19333976, 4)


In [22]:
from lsstdesc_diffsky.photometry.get_SFH_from_params import get_sfh_from_params
sfh_red = get_sfh_from_params(mah_params[red_mask], ms_u_params[red_mask], q_u_params[red_mask], LGT0, times)
sfh_blue = get_sfh_from_params(mah_params[blue_mask], ms_u_params[blue_mask], q_u_params[blue_mask], LGT0, times)
print(sfh_red.shape, sfh_blue.shape)

.......computing SFHs from diffmah/star params for 100 times
.......using parameters with shapes (746, 4), (746, 5), (746, 4)
.......computing SFHs from diffmah/star params for 100 times
.......using parameters with shapes (337, 4), (337, 5), (337, 4)
(746, 100) (337, 100)


# Now compute some sample SEDs